# Brian's Notebook for Project 1

In [1]:
import pandas as pd

def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

# Select weeks to aggregate data by YYMMDD format
week_nums = [200627, 200620, 200613]

In [2]:
# Define 'df' as variable for exported data
df = get_data(week_nums)

In [3]:
df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,00:00:00,REGULAR,7424218,2522558
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,04:00:00,REGULAR,7424220,2522559
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,08:00:00,REGULAR,7424231,2522572
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,12:00:00,REGULAR,7424265,2522590
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,16:00:00,REGULAR,7424340,2522604
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,20:00:00,REGULAR,7424415,2522612
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/21/2020,00:00:00,REGULAR,7424441,2522622
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/21/2020,04:00:00,REGULAR,7424443,2522623
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/21/2020,08:00:00,REGULAR,7424457,2522628
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/21/2020,12:00:00,REGULAR,7424471,2522645


In [4]:
df.tail(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
206652,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/11/2020,09:00:00,REGULAR,5554,514
206653,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/11/2020,13:00:00,REGULAR,5554,514
206654,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/11/2020,17:00:00,REGULAR,5554,514
206655,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/11/2020,21:00:00,REGULAR,5554,514
206656,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,01:00:00,REGULAR,5554,514
206657,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,05:00:00,REGULAR,5554,514
206658,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,09:00:00,REGULAR,5554,514
206659,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,13:00:00,REGULAR,5554,514
206660,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,17:00:00,REGULAR,5554,514
206661,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,21:00:00,REGULAR,5554,514


In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
# Average of all entries
df.ENTRIES.mean()

43205359.19525636

In [7]:
# Total counts of when the STATION is listed
df.STATION.value_counts()

34 ST-PENN STA     12616
FULTON ST          12093
GRD CNTRL-42 ST     9580
23 ST               9170
86 ST               7923
                   ...  
190 ST               375
NORWOOD AV           374
ATLANTIC AV          370
SUTTER AV            252
CLEVELAND ST         252
Name: STATION, Length: 379, dtype: int64

In [8]:
df[df.ENTRIES > df.ENTRIES.mean()].head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
924,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,02:00:00,REGULAR,119672719,3234056
925,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,06:00:00,REGULAR,119672720,3234056
926,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,10:00:00,REGULAR,119672722,3234069
927,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,14:00:00,REGULAR,119672735,3234097
928,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,18:00:00,REGULAR,119672774,3234139
929,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/20/2020,22:00:00,REGULAR,119672812,3234152
930,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/21/2020,02:00:00,REGULAR,119672820,3234155
931,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/21/2020,06:00:00,REGULAR,119672821,3234155
932,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/21/2020,10:00:00,REGULAR,119672824,3234168
933,A007,R079,01-06-01,5 AV/59 ST,NQRW,BMT,06/21/2020,14:00:00,REGULAR,119672832,3234197


In [9]:
# There is no missing data
df.isna().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

In [10]:
# Aggregation of ENTRIES by STATION; could not aggregate by EXITS; detailed
df.groupby('STATION').ENTRIES.agg(['mean', 'std', 'median', 'count'])

,mean,std,median,count
STATION,,,,
1 AV,2.307557e+07,8.094204e+07,104980.0,2519
103 ST,9.000996e+06,7.210597e+06,8625196.0,1896
103 ST-CORONA,6.917390e+06,5.291068e+06,5030881.0,1134
104 ST,1.879381e+08,5.281403e+08,401215.5,1134
110 ST,6.772439e+06,3.494255e+06,5485303.0,755
...,...,...,...,...
WOODLAWN,6.463689e+06,3.127540e+06,8107492.0,630
WORLD TRADE CTR,8.897771e+07,3.617463e+08,663148.0,2268
WTC-CORTLANDT,2.111268e+07,1.218744e+08,144506.0,4401


In [11]:
# Aggregation of ENTRIES by STATION; could not aggregate by 
df.groupby('STATION')[['ENTRIES']].sum()

,ENTRIES
STATION,
1 AV,58127365579
103 ST,17065888283
103 ST-CORONA,7844320158
104 ST,213121779869
110 ST,5113191406
...,...
WOODLAWN,4072124304
WORLD TRADE CTR,201801446100
WTC-CORTLANDT,92916914006


In [12]:
# Aggregation of ENTRIES by TIME; unsuccessful
df.groupby('TIME')[['ENTRIES']].sum()

,ENTRIES
TIME,
00:00:00,2604031692254
00:00:02,506293
00:00:07,413824
00:00:10,308213
00:00:11,1
...,...
23:59:47,57316
23:59:50,1921
23:59:51,34984


**From the above, we can see that there is an issue with aggregating data by time. In an effort to remedy this, adjust the time from ##:##:## format to ##.**

In [13]:
# Create a lambda function to remove 6 characters from end of string
new_time = df.TIME.map(lambda x: str(x)[:-6])

df['ABBREV_TIME'] = new_time
df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ABBREV_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,00:00:00,REGULAR,7424218,2522558,00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,04:00:00,REGULAR,7424220,2522559,04
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,08:00:00,REGULAR,7424231,2522572,08
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,12:00:00,REGULAR,7424265,2522590,12
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/20/2020,16:00:00,REGULAR,7424340,2522604,16
...,...,...,...,...,...,...,...,...,...,...,...,...
206657,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,05:00:00,REGULAR,5554,514,05
206658,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,09:00:00,REGULAR,5554,514,09
206659,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,13:00:00,REGULAR,5554,514,13
206660,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/12/2020,17:00:00,REGULAR,5554,514,17


In [14]:
# Aggregation of ENTRIES by ABBREV_TIME; it works!
df.groupby('ABBREV_TIME')[['ENTRIES']].sum()

,ENTRIES
ABBREV_TIME,
00,2734797261771
01,1545344114618
02,85198410177
03,84480122933
04,2739778702066
05,1563644155027
06,85360532054
07,88649188024
08,2736657058686
